In [2]:
%%time
def dir_padding(i):
    if i < 10: return '000' + str(i)
    elif i < 100: return '00' + str(i)
    elif i < 1000: return '0' + str(i)
    elif i < 10000 : return str(i)

Wall time: 0 ns


In [3]:
def file_padding(i):
    if i < 10: return '00000' + str(i)
    elif i < 100: return '0000' + str(i)
    elif i < 1000: return '000' + str(i)
    elif i < 10000: return '00' + str(i)
    elif i < 100000: return '0' + str(i)
    else: return str(i)

In [4]:
def get_path(base_path, base_dir, mid_dir, filename):
    return base_path + base_dir + '/' + mid_dir + '/' + filename

In [5]:
from collections import Counter, defaultdict
def preprocess_label(sentence, dict_)
    for ch in sentence:
        if ch == ' ':continue
        dict_is_all_you_need[ch] += 1

    keys = dict_.keys()
    values = dict_.values()

    values, keys = zip(*sorted(zip(values, keys), reverse=True))

SyntaxError: invalid syntax (<ipython-input-5-e7125d8a7667>, line 2)

In [6]:
%%time
EXCEPTION = ['(', ')', '/']
folder_change_point = [124 * x for x in range(1,5)]
print(folder_change_point)
base_path = "E:/한국어 음성데이터/"
base_dir = ['KsponSpeech_0' + str(x) for x in range(1,6)]
print(base_dir)
mid_dir = ['KsponSpeech_' + str(dir_padding(x)) for x in range(1,624)]

base_dir_idx = 0
mid_dir_idx = 0
past_mid_dir_idx = -1
print('started...')
for file_num in range(1, 622546):
    if file_num % 1000 == 1 and file_num != 1: mid_dir_idx += 1
    if mid_dir_idx in folder_change_point and mid_dir_idx != past_mid_dir_idx:
        past_mid_dir_idx = mid_dir_idx
        base_dir_idx += 1
    path = get_path(base_path, base_dir[base_dir_idx], mid_dir[mid_dir_idx], 'KsponSpeech_' + file_padding(file_num))
    file = open(path, 'r')
    file.close()
print('ended!!!')

[124, 248, 372, 496]
['KsponSpeech_01', 'KsponSpeech_02', 'KsponSpeech_03', 'KsponSpeech_04', 'KsponSpeech_05']
started...


FileNotFoundError: [Errno 2] No such file or directory: 'E:/한국어 음성데이터/KsponSpeech_01/KsponSpeech_0001/KsponSpeech_000001'

In [4]:
text = "아 씨발 (2019년)/(이천 십 구 년) 다 갔네 b/"
text2 = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
text3 = "근데 (3학년)/(삼 학년) 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?"
def filter_double_copied(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

filter_double_copied(text3)

'근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?'

In [12]:
SENTENCE_MARK = ['.', '?', ',', '!']
def del_special_ch(sentence):
    import re
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            if idx + 1 < len(sentence) and ch.isalpha() and sentence[idx+1] == '/': continue
        if ch != '/' and ch != '+' and ch != '*': new_sentence += ch
    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence
del_special_ch("근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?")

'근데 삼 학년 때 까지는 국가장학금 바 받으면서 다녔던 건가?'

In [6]:
def sentence_filter(raw_sentence):
    return del_special_ch(filter_double_copied(raw_sentence))
sentence_filter("근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?")

'근데 삼 학년 때 까지는 국가장학금 바 받으면서 다녔던 건가?'

In [7]:
%%time
import os
from tqdm import trange
EXCEPTION = ['(', ')', '/']
folder_change_point = [124 * x for x in range(1,5)]
base_path = "E:/한국어 음성데이터/"
base_dir = ['KsponSpeech_0' + str(x) for x in range(1,6)]
mid_dir = ['KsponSpeech_' + str(dir_padding(x)) for x in range(1,624)]

base_dir_idx = 0
mid_dir_idx = 0
past_mid_dir_idx = -1
label_list = list()
label_freq = list()
cnt = 0
print('started...')
for file_num in trange(1, 622546):
    if file_num % 1000 == 1 and file_num != 1: mid_dir_idx += 1
    if mid_dir_idx in folder_change_point and mid_dir_idx != past_mid_dir_idx:
        past_mid_dir_idx = mid_dir_idx
        base_dir_idx += 1
    path = get_path(base_path, base_dir[base_dir_idx], mid_dir[mid_dir_idx], 'KsponSpeech_' + file_padding(file_num))
    file = open(path + ".txt", 'r')
    text = file.readline()
    for ch in text:
        if ch not in label_list:
            label_list.append(ch)
            label_freq.append(1)
        else:
            label_freq[label_list.index(ch)] += 1

label_freq, label_list = zip(*sorted(zip(label_freq, label_list), reverse=True))
print('ended!!!')

started...


  0%|                                                                                       | 0/622545 [00:00<?, ?it/s]

아/ 몬 소리야, 그건 또. b/
나는 악습은 원래 없어진다+ 없어져야 된다고 생각하긴 했는데 근데 그/ 약간 필요악으로 하나 정도쯤은 있어야 되거든. 물 뜨러 가고.

b/ n/ 그래서 지호랑 계단 n/ 올라와서 b/ 막 위에 운동하는 기구 있대요. b/ 그서 그걸로 운동 할려구요. b/ n/

뭐/ 정신과 병원도 그 약 타서 먹어보고, 그 한동안 연락이 안 된 적이 있었단 말이야. 그때가 언제였 언제였더라?

o/ b/ 그게 (0.1프로)/(영 점 일 프로) 가정의 아이들과 가정의 모습이야? b/

그/ 친애하는 판사님께라는 법+ 법 관련 드라마 알고 있어?

o/ 그래가지고 진짜 차 사야겠다 아니 뭐/ 차 안 되면 스쿠터라도 타야되겠다 막/ 그런 생각 들더라구 그래서 운전은 하는 게 좋은 거 같애 진짜 b/

그래

o/ 나도 몰라. 나 그/ (3G)/(쓰리 쥐)* 하나도 안 봤음. 어.

아/ 내일 나 알바하구나.

n/ 아/ 근데 (1시)/(한 시)에 닫는 게 쫌 아쉽긴 한데 거기 진짜 괜찮은데 b/

맞아. 시간 안에 풀고 약간 이것부터 풀고 요런 식으로 풀어보렴. 이런 거 b/

삼 점대야? 언니가?

한+ 한+ 한 시간에 이 만 원? 거의 이 정도로 이 정도란 말이야. b/

굳이 싶기도 해. 까* 오히려 사원 이런 것보다는 b/ 대신 돔* 먹+ 먹, 음식 거리, 먹거리가 우리 입맛에 좀 더 맞는 거 같고.

아/ 근데 강남 너무 비싸.

나, 먼저 집에 가께

o/ l/ 해? 아/ 뭔가 효과가 있어 근데?

n/ 근데 이+ 나만 안 하는 거 같애. 딴 사람들은 다 하고 l/

o/ 아 b/ 설 연휴 때? 집에 있겠네 그럼 거의? b/ 음 그 옆집 사는 애?

왜 b/

무한도전은 완전 폐지돼서 없어졌는데 너는 무한도전을 자주 잘 챙겨 봤어?

o/ 나 다음 주에 강릉 가는데 같이 가자. b/ 야, 너 근데 학교 기숙사에서 살면 안 되냐?

목포 너무 비싸, b/ 어, 시외버스도 비싸고, 한 (3만원)/(삼만 원)인가 그럴걸.



  0%|                                                                             | 24/622545 [00:00<43:58, 235.92it/s]

근데 왜 그 집을 기억을 못 해?

복싱 안 배워봤어?

o/ 아/ 니가 사주는 거였어?

n/ 과학적 근거가 있는 팩튼지

알겠어.

그/ 카레. 뿌라뽕 커리죠. 그/ 카레랑 같이 이렇게 소스만 만드는 법을 좀 배우면 한번 시도해볼 수도 있지 않을까 싶어요. 특히 좀 겨울철에 먹기에 굉장히 좋은 음식인 거 같아요.

o/ 아/ 그건 절대로 안 할 거야.

그렇긴 했는데

아 진짜 그러 에이씨 좋은 학교 다니니까 b/ 그런 것도 배우고 부럽다

l/ 불편하잖아, 장갑.

응 근데 이거 한 세 시에 끝나니까.

글쎄.

할머니가 아까 너가 말한 거 같은 복숭아 맛 그런 요플레 l/ 그걸로 해서 드셨던 거 같애.

o/ b/ 네+ 네. b/

o/ b/ 어 배우고 연습+ 연습할 시간이 없는데 어떻게 연습을 해요? 라고 여기까지 찼지만 b/ 아/ 네 할게요로 눌러버+ 눌러버렸지. b/

o/ b/ 대단하다 그런 생각, 난 생각 조차 못 하겠더라. 나는

o/ 그래. 어차피 먹어야 되니까 인제 음/

b/ 빨리 가야 돼.

헬스는 끝났어?

아니야 좀 심했어, 진짜.

불가리아 그쪽인가?

전문가용인데, 그/ 막 음너+ 음원 녹음하고 그런 거 올리는 b/ 그런 아이딘가 봐 나 근데 나 잘 모르겠어.



  0%|                                                                             | 46/622545 [00:00<45:19, 228.94it/s]

o/ 왜 자랐냐고?

재수를 마치고 지금은 알바를 찾고 있는 거 같아.

o/ 아이* 근데 b/ 나는 너무 억지 영화* 억지스러운 웃음 유발. 그래가주고 천만 난 갈+ b/ 간다고 생각 안 했거든.

b/ 뭐 생각해 논 거 있어? b/

투표를 뭐/ 자주 할지언정

o/ 아이/ 말해봐.

l/ b/ 그거 찍을 생각을 못 했어.

근데 진짜 잘해.

n/ 웃기지 마. 니 l/ 니 제일 좋아했잖아. 내 썰 듣는 거.

얘는 바쁘구 나는 여유롭다고 여유로운 사람이 매달리고 어/ 나랑 놀자 이러고 그건 좀 자기 욕심이야. 진짜 자기만 생각하는 거야.

o/ 몰라 아직도 이유를 모르겠어.

맞아. 나도 (2015년)/(이천 십 오 년)에 프로필 찍었을 때 b/ 정말 나 혼자 가고 처음 찍는 건데 아예 생판 모르는 포토그래퍼한테 찍어가지고 b/ 어/ 괜찮긴 한데 너무 사진들을 지금 보면 얼굴에 어색함이 b/ 많아가지고 어따 쓸 수도 없고 b/ 컨셉 회의 같은 걸 안 했었어. 근데 나는 그런 걸 원래 하는지 몰랐어.

아니, 난 모르겠더라.

빨리 일정이 나오면 그래도 어느 정돈 하는데.

당연하지. 우리 아파트 살잖아.

아이/ 딱히 효과 없어가지고 안에서 바를 때도.

바로 월정리 해수욕장 앞에.

어 막 트라이팟이 이런 거 좀 그렇게 보이잖아. 어.

o/ 어 그러지 말라 그래 나 진짜 근데 이번에 또 오면 진짜 차단할 거야.

응/ 니가 바리깡 가져오라고 재촉해 가지고 바리깡만 챙겼어. l/

어쨌든 걔네+ 걔네랑 이제 내일 오랜만에 만나가지고 밥 먹기로 했는데

o/ 응. b/ 그래+ 아/ 까 b/ 아마 좀 있으면 괜찮아질 거야 이랬거든. 근데 지금 한지 좀 지났는데 아직도 눈이 시리대. 그래서* 병원에 가서 얘길 했더니 b/ 난시가 너무 심한 눈이어서 이게 꽤 오래갈 수도 있다는 거야. b/

b/ 아니야 나, 내가 농구경기 직관 한 번 가봤는데, 밤에 했어.

o/ 근데 대+ 댓글 보니까. 땅콩 드리겠지. 뭐. 이러던데? 어/ 대한 항공. b/

야 아직 어리

  0%|                                                                             | 73/622545 [00:00<43:33, 238.16it/s]

o/ 잠만 기달려봐, 지금. (11시)/(열 한 시). 어차피 니 (11시)/(열 한 시) (20분)/(이십 분)까지 도착해야 되는 거지?

아/ 그래서 팔팔이 (1컷)/(일 컷)이 나왔구나. 우리 아이/ 내가 보통 비킬러 풀면은 이/ 비킬러 세무 그에가 킬러 세 문제 빼면은 스물 일곱 문제잖아? 내 스물 일곱 문제 푸는 나는 보통 한 (2)/(이), (30분)/(삼십 분) 걸리고 거기다 (10)/(십 분)?

o/ 요기 살아. n/ 여기 그/ 어디지? 상수동.

난 몰라. 뭐하는 덴지

o/ 경영학관데, 연대 경제 뚫었어, 편입으로. 나 진짜* 그래서, b/ 에타 학게가* 있었어. 탈홍합니다. b/

진에어나 이런 거 타봤어?

근데 저 명지대 다니는데 학교 주변에 막 이대, 연대, 홍대, 서강대 이렇잖아요. 그래서 그런 애들 엄청 무시하더라고요.

대강 그렇게 해서 치면은 칠만 한 거 같던데

아무튼 순우형.

그래 지금 많이 시도 엄마한테 그/ 어머니한테 그때 전화해 가지고 b/

나 그걸로 요즘 미생 봐. 아/ 나 요즘 미생 봐, 미생. 어.

n/ 있잖아. n/ 그/ 너 해외여행 간 적 있어?

n/ 재수할 때 많이 와봤지. 여기는.

그거야 모르지.

o/ 오/ 신기하다. 음 무슨 알바를 그렇게 신중하게

o/ 응.

애들 막/ 못 막잖아.

n/ 금방 가. 네 시까지. n/

그 외에는 또 뭐 다른 게, 만들어 먹던 그런 건 없었어?

아/ (13일)/(십 삼 일). 너 (13일)/(십 삼 일)날 뭐 한다 했지? n/

어. 바로+ 바로 이적했지.

b/ 야 그만 웃으라고.

o/ 근데 우리 신랑이 그런 데만 가면 b/ 그/ 투어버스를 너무 좋아해.

예전에는 B 언어였는데. 지금은 C 데, C언어부터 기반을 다져서 게임은 C샾 이라는 걸 써.

n/ 월남쌈 안 먹은 지 오래됐는데. b/

o/ n/ b/ 쌔 차 사니까 아빠가 맨날 자랑해. b/ 등에서 히터 나오고 b/ 엉덩이에서 히터 나오고 막/ b/ 무슨 기능들이 뭐가 이렇게 많다고 설명하는데

ValueError: not enough values to unpack (expected 2, got 0)

In [11]:
label_list

(' ',
 '.',
 '그',
 '이',
 '는',
 '가',
 '고',
 '어',
 '아',
 '거',
 '지',
 '데',
 '?',
 '나',
 '하',
 '다',
 '서',
 '에',
 '도',
 '게',
 '니',
 ',',
 '기',
 '은',
 '면',
 '야',
 '있',
 '을',
 '한',
 '까',
 '해',
 '리',
 '라',
 '래',
 '사',
 '근',
 '들',
 '안',
 '로',
 '일',
 '내',
 '보',
 '제',
 '같',
 '자',
 '만',
 '시',
 '런',
 '너',
 '대',
 '때',
 '되',
 '으',
 '뭐',
 '진',
 '를',
 '잖',
 '러',
 '인',
 '오',
 '무',
 '었',
 '구',
 '했',
 '수',
 '간',
 '애',
 '우',
 '요',
 '생',
 '마',
 '렇',
 '냥',
 '짜',
 '주',
 '없',
 '말',
 '학',
 '스',
 '더',
 '많',
 '원',
 '정',
 '겠',
 '먹',
 '여',
 '금',
 '든',
 '부',
 '할',
 '막',
 '전',
 '번',
 '좋',
 '랑',
 '네',
 '람',
 '약',
 '건',
 '각',
 '음',
 '좀',
 '알',
 '잘',
 '걸',
 '모',
 '것',
 '상',
 '려',
 '장',
 '히',
 '않',
 '맞',
 '던',
 '르',
 '교',
 '바',
 '드',
 '냐',
 '십',
 '날',
 '치',
 '비',
 '단',
 '동',
 '또',
 '못',
 '얘',
 '중',
 '저',
 '의',
 '난',
 '엄',
 '봤',
 '걔',
 '화',
 '싶',
 '갔',
 '았',
 '집',
 '왜',
 '계',
 '공',
 '긴',
 '신',
 '적',
 '연',
 '직',
 '실',
 '영',
 '미',
 '응',
 '봐',
 '분',
 '테',
 '년',
 '트',
 '문',
 '돼',
 '물',
 '예',
 '터',
 '세',
 '럼',
 '청',
 '차',
 '친'

In [12]:
label_freq

1

In [16]:
import pandas as pd
label = {'id':[], 'char':[], 'freq' :[]}
for idx, ch_freq in enumerate(zip(label_list, label_freq)):
    ch, freq = ch_freq
    label['id'].append(idx)
    label['char'].append(ch)
    label['freq'].append(freq)
df_label = pd.DataFrame(label)
df_label.to_csv('./aihub_labels.csv', encoding='utf-8', index=False)

AttributeError: module 'pandas' has no attribute '_libs'

In [17]:
import pickle

data = {
    'label_list': label_list,
    'label_freq': label_freq
}

# save
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [18]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [27]:
label_list = data['label_list']

In [28]:
label_list

(' ',
 '.',
 '그',
 '이',
 '는',
 '가',
 '고',
 '어',
 '아',
 '거',
 '지',
 '데',
 '?',
 '나',
 '하',
 '다',
 '서',
 '에',
 '도',
 '게',
 '니',
 ',',
 '기',
 '은',
 '면',
 '야',
 '있',
 '을',
 '한',
 '까',
 '해',
 '리',
 '라',
 '래',
 '사',
 '근',
 '들',
 '안',
 '로',
 '일',
 '내',
 '보',
 '제',
 '같',
 '자',
 '만',
 '시',
 '런',
 '너',
 '대',
 '때',
 '되',
 '으',
 '뭐',
 '진',
 '를',
 '잖',
 '러',
 '인',
 '오',
 '무',
 '었',
 '구',
 '했',
 '수',
 '간',
 '애',
 '우',
 '요',
 '생',
 '마',
 '렇',
 '냥',
 '짜',
 '주',
 '없',
 '말',
 '학',
 '스',
 '더',
 '많',
 '원',
 '정',
 '겠',
 '먹',
 '여',
 '금',
 '든',
 '부',
 '할',
 '막',
 '전',
 '번',
 '좋',
 '랑',
 '네',
 '람',
 '약',
 '건',
 '각',
 '음',
 '좀',
 '알',
 '잘',
 '걸',
 '모',
 '것',
 '상',
 '려',
 '장',
 '히',
 '않',
 '맞',
 '던',
 '르',
 '교',
 '바',
 '드',
 '냐',
 '십',
 '날',
 '치',
 '비',
 '단',
 '동',
 '또',
 '못',
 '얘',
 '중',
 '저',
 '의',
 '난',
 '엄',
 '봤',
 '걔',
 '화',
 '싶',
 '갔',
 '았',
 '집',
 '왜',
 '계',
 '공',
 '긴',
 '신',
 '적',
 '연',
 '직',
 '실',
 '영',
 '미',
 '응',
 '봐',
 '분',
 '테',
 '년',
 '트',
 '문',
 '돼',
 '물',
 '예',
 '터',
 '세',
 '럼',
 '청',
 '차',
 '친'

In [22]:
data

{'label_list': (' ',
  '.',
  '그',
  '이',
  '는',
  '가',
  '고',
  '어',
  '아',
  '거',
  '지',
  '데',
  '?',
  '나',
  '하',
  '다',
  '서',
  '에',
  '도',
  '게',
  '니',
  ',',
  '기',
  '은',
  '면',
  '야',
  '있',
  '을',
  '한',
  '까',
  '해',
  '리',
  '라',
  '래',
  '사',
  '근',
  '들',
  '안',
  '로',
  '일',
  '내',
  '보',
  '제',
  '같',
  '자',
  '만',
  '시',
  '런',
  '너',
  '대',
  '때',
  '되',
  '으',
  '뭐',
  '진',
  '를',
  '잖',
  '러',
  '인',
  '오',
  '무',
  '었',
  '구',
  '했',
  '수',
  '간',
  '애',
  '우',
  '요',
  '생',
  '마',
  '렇',
  '냥',
  '짜',
  '주',
  '없',
  '말',
  '학',
  '스',
  '더',
  '많',
  '원',
  '정',
  '겠',
  '먹',
  '여',
  '금',
  '든',
  '부',
  '할',
  '막',
  '전',
  '번',
  '좋',
  '랑',
  '네',
  '람',
  '약',
  '건',
  '각',
  '음',
  '좀',
  '알',
  '잘',
  '걸',
  '모',
  '것',
  '상',
  '려',
  '장',
  '히',
  '않',
  '맞',
  '던',
  '르',
  '교',
  '바',
  '드',
  '냐',
  '십',
  '날',
  '치',
  '비',
  '단',
  '동',
  '또',
  '못',
  '얘',
  '중',
  '저',
  '의',
  '난',
  '엄',
  '봤',
  '걔',
  '화',
  '싶',
  '갔',
  '았',
  '집',
  '왜',